## 读取数据

In [1]:
# 首先 import 必要的模块
import pandas as pd #数据分析
import numpy as np #科学计算
from pandas import Series,DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
#打开文件
# path to where the data lies
dpath = './input/'
train = pd.read_csv(dpath +'train_data_org.csv',index_col='用户编码')
y_train=train.pop('信用分')
X_train=train.drop(['data_type'], axis=1)

In [2]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
def score(y_pred, y_true):
    return 1/(mean_absolute_error(y_pred, y_true)+1)
My_Score = make_scorer(score, greater_is_better=True)

## XGBRegressor

### 1.最佳迭代次数：n_estimators

In [3]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
cv_params = {'n_estimators': [4000]}
other_params = {'learning_rate': 0.01, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'objective':'reg:gamma',
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
model = xgb.XGBRegressor(**other_params)
Grid = GridSearchCV(estimator=model, param_grid=cv_params, scoring=My_Score, cv=5, verbose=3)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] n_estimators=4000 ...............................................


D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV] ..... n_estimators=4000, score=0.06305915049136063, total= 6.3min
[CV] n_estimators=4000 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.5min remaining:    0.0s
D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV] ..... n_estimators=4000, score=0.06353346429132468, total= 6.3min
[CV] n_estimators=4000 ...............................................


...

[CV] ..... n_estimators=4000, score=0.06363300101654923, total= 6.3min
[CV] n_estimators=4000 ...............................................


D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV] ..... n_estimators=4000, score=0.06355275670966569, total= 6.3min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 32.6min finished
D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


每轮迭代运行结果:[0.06367945]
参数的最佳取值：{'n_estimators': 4000}
最佳模型得分:0.0636794539659308


In [4]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
cv_params = {'n_estimators': [3000]}
other_params = {'learning_rate': 0.01, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'objective':'reg:gamma',
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
model = xgb.XGBRegressor(**other_params)
Grid = GridSearchCV(estimator=model, param_grid=cv_params, scoring=My_Score, cv=5, verbose=3, n_jobs=4)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed: 12.5min remaining: 18.7min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed: 17.5min finished


每轮迭代运行结果:[0.06361431]
参数的最佳取值：{'n_estimators': 3000}
最佳模型得分:0.06361431260361612


### 2.调试的参数:min_child_weight以及max_depth

##### 经筛选n_estimators最佳为315

In [5]:
cv_params = {'max_depth': [7], 'min_child_weight': [1]}
other_params = {'learning_rate': 0.01, 'n_estimators': 4000, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'objective':'reg:gamma',
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
model = xgb.XGBRegressor(**other_params)
Grid = GridSearchCV(estimator=model, param_grid=cv_params, scoring=My_Score, cv=5, verbose=3)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] max_depth=7, min_child_weight=1 .................................


D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  max_depth=7, min_child_weight=1, score=0.0630580589870324, total= 9.9min
[CV] max_depth=7, min_child_weight=1 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 10.4min remaining:    0.0s
D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  max_depth=7, min_child_weight=1, score=0.06340908113767002, total= 9.0min
[CV] max_depth=7, min_child_weight=1 .................................


...

[CV]  max_depth=7, min_child_weight=1, score=0.06354710907469475, total= 9.0min
[CV] max_depth=7, min_child_weight=1 .................................


D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  max_depth=7, min_child_weight=1, score=0.06336110064345174, total= 9.0min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 48.6min finished
D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


每轮迭代运行结果:[0.06359221]
参数的最佳取值：{'max_depth': 7, 'min_child_weight': 1}
最佳模型得分:0.06359220501146765


In [4]:
cv_params = {'max_depth': [4,5,6], 'min_child_weight': [0.9,1]}
other_params = {'learning_rate': 0.01, 'n_estimators': 4000, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'objective':'reg:gamma',
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
model = xgb.XGBRegressor(**other_params)
Grid = GridSearchCV(estimator=model, param_grid=cv_params, scoring=My_Score, cv=5, verbose=3)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] max_depth=4, min_child_weight=0.9 ...............................


D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  max_depth=4, min_child_weight=0.9, score=0.06297467416496062, total= 5.4min
[CV] max_depth=4, min_child_weight=0.9 ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.5min remaining:    0.0s
D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  max_depth=4, min_child_weight=0.9, score=0.06350054988156603, total= 5.5min
[CV] max_depth=4, min_child_weight=0.9 ...............................


...

[CV]  max_depth=6, min_child_weight=1, score=0.06372565814168475, total= 7.9min
[CV] max_depth=6, min_child_weight=1 .................................


D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[CV]  max_depth=6, min_child_weight=1, score=0.06354561795244132, total= 8.0min


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 212.2min finished
D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


每轮迭代运行结果:[0.06358287 0.06357966 0.06368013 0.06367945 0.06368781 0.06368966]
参数的最佳取值：{'max_depth': 6, 'min_child_weight': 1}
最佳模型得分:0.06368965574579563


In [14]:
cv_params = {'max_depth': [5], 'min_child_weight': [1]}
other_params = {'learning_rate': 0.1, 'n_estimators': 320, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'objective':'reg:gamma',
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
model = xgb.XGBRegressor(**other_params)
Grid = GridSearchCV(estimator=model, param_grid=cv_params, scoring=My_Score, cv=5, verbose=3)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.5min remaining:  2.3min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  2.2min finished


每轮迭代运行结果:[0.06330317]
参数的最佳取值：{'max_depth': 5, 'min_child_weight': 1}
最佳模型得分:0.0633031734448777


### 3.调试参数：gamma

##### n_estimator：315   max_depth：5   min_child_weight:1

In [15]:
cv_params = {'gamma': [0.000000001]}
other_params = {'learning_rate': 0.1, 'n_estimators': 320, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'objective':'reg:gamma',
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}
model = xgb.XGBRegressor(**other_params)
Grid = GridSearchCV(estimator=model, param_grid=cv_params, scoring=My_Score, cv=5, verbose=3, n_jobs=4)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.6min remaining:  2.4min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  2.4min finished


每轮迭代运行结果:[0.06330317]
参数的最佳取值：{'gamma': 1e-09}
最佳模型得分:0.0633031734448777


### 4.调试参数：subsample以及colsample_bytree

##### n_estimator：315    max_depth：5     min_child_weight:1     gamma:0.000000001

In [16]:
cv_params = { 'colsample_bytree': [0.7],'subsample': [0.93]}
other_params = {'learning_rate': 0.1, 'n_estimators': 315, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'objective':'reg:gamma',
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.000000001, 'reg_alpha': 0, 'reg_lambda': 1}
model = xgb.XGBRegressor(**other_params)
Grid = GridSearchCV(estimator=model, param_grid=cv_params, scoring=My_Score, cv=5, verbose=3, n_jobs=4)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   5 | elapsed:  1.3min remaining:  2.0min
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:  2.2min finished


每轮迭代运行结果:[0.06338015]
参数的最佳取值：{'colsample_bytree': 0.7, 'subsample': 0.93}
最佳模型得分:0.06338014793595492


### 5.调试参数：reg_alpha以及reg_lambda：

##### 'n_estimator'：315    'max_depth'：5     'min_child_weight':1     'gamma':0.000000001    'colsample_bytree': 0.7     'subsample': 0.93

In [3]:
cv_params = {'reg_alpha': [0.000001,0.0001 ], 'reg_lambda': [1]}
other_params = {'learning_rate': 0.1, 'n_estimators': 320, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'objective':'reg:gamma',
                    'subsample': 0.93, 'colsample_bytree': 0.7, 'gamma':0.000000001, 'reg_alpha': 0, 'reg_lambda': 1}
model = xgb.XGBRegressor(**other_params)
Grid = GridSearchCV(estimator=model, param_grid=cv_params, scoring=My_Score, cv=5, verbose=3, n_jobs=4)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:  3.7min remaining:  1.6min
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  6.3min finished


每轮迭代运行结果:[0.06338181 0.06332858]
参数的最佳取值：{'reg_alpha': 1e-06, 'reg_lambda': 1}
最佳模型得分:0.06338181086393473


### 6.调试参数：learning_rate

##### 'n_estimator'：320    'max_depth'：5     'min_child_weight':1     'gamma':0.000000001    
##### 'colsample_bytree': 0.7     'subsample': 0.93   'reg_alpha': 0.0001, 'reg_lambda': 1

In [6]:
cv_params = {'learning_rate': [ 0.07, 0.1, 0.11]}
other_params = {'learning_rate': 0.1, 'n_estimators': 320, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'objective':'reg:gamma',
                    'subsample': 0.93, 'colsample_bytree': 0.7, 'gamma': 0.000000001, 'reg_alpha': 0.000001, 'reg_lambda': 1}
model = xgb.XGBRegressor(**other_params)
Grid = GridSearchCV(estimator=model, param_grid=cv_params, scoring=My_Score, cv=5, verbose=3, n_jobs=4)
Grid.fit(X_train, y_train)
evalute_result = Grid.cv_results_[ 'mean_test_score' ]
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(Grid.best_params_))
print('最佳模型得分:{0}'.format(Grid.best_score_))

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  15 out of  15 | elapsed: 10.5min finished


每轮迭代运行结果:[0.06334859 0.06338181 0.06328552]
参数的最佳取值：{'learning_rate': 0.1}
最佳模型得分:0.06338181086393473


##### 'n_estimator'：315    'max_depth'：5     'min_child_weight':1     'gamma':0.000000001    
##### 'colsample_bytree': 0.7     'subsample': 0.93   'reg_alpha': 0.0001, 'reg_lambda': 1  'learning_rate': 0.07

## 测试集

In [ ]:
import xgboost as xgb
other_params = {'learning_rate': 0.07, 'n_estimators': 315, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,'objective':'reg:gamma',
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0.0001, 'reg_lambda': 1}
model = xgb.XGBRegressor(**other_params)
model.fit(X_train, y_train)

In [35]:
#打开文件
# path to where the data lies
dpath = './input/'
test = pd.read_csv(dpath +'test_data_org.csv',index_col='用户编码')
X_test=test.drop(['data_type'], axis=1)
X_test_index=X_test.index
#y预测，四舍五入，取整
y_pred=np.around(model.predict(X_test)).astype(int)
#生成提交结果
user_id=pd.DataFrame(X_test_index).rename(columns = {'用户编码':'id'})
score = pd.DataFrame(y_pred,columns=['score'])
out_df = pd.concat([user_id,score], axis = 1)
out_df.to_csv("./output/ElasticNet.csv", index=False)